In [1]:
import re
import sys
import os
from pathlib import Path


from PRINT_data_utilities import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

In [39]:
class Ui_FileDialog(object):
    def setupUi(self, Dialog):
        if not Dialog.objectName():
            Dialog.setObjectName(u"Dialog")
        Dialog.resize(1000,620)
        # Dialog.setStyleSheet(u"background-color: #5d707f")

class FileDialog(QFileDialog,Ui_FileDialog):
    """  default dialog class for PRINT_py with OK and ABBRECHEN buttons and setable text """

    def __init__(self,title = 'default window',parent=None):
        super().__init__(parent)
        
        self.setupUi(self)
        self.setWindowTitle(title)
        self.setFileMode(1)     # enum for "ExistingFile"
        self.setDirectory(r"C:\Users\Max\Desktop\MultiCarb3D\CAD2RAPID")
        self.setNameFilters({"GCode files (*.gcode)",
                             "RAPID files (*.mod)"})
        self.setStyleSheet("QWidget {font-family: \"Bahnschrift\"; font-size: 10pt; background-color: #f2f4f3;} \
                            QPushButton {background-color: #FFBA00; border-radius: 5px; font-size: 14pt; min-width: 100px;} \
                            QPushButton::hover {background-color: #FFC300;} \
                            QPushButton::pressed {background-color: #88AB75;} \
                            ")
        
        # uncomment here if you want a custom sidebar
        # self.setOption(QFileDialog.DontUseNativeDialog,on = True)
        # self.setSidebarUrls([QUrl.fromLocalFile("/Users/Max/Desktop/MultiCarb3D/CAD2RAPID"),
        #                      QUrl.fromLocalFile(QStandardPaths.writableLocation(QStandardPaths.DesktopLocation)),
        #                      QUrl.fromLocalFile(QStandardPaths.writableLocation(QStandardPaths.RuntimeLocation))])


app = 0
app = QApplication(sys.argv)
win = FileDialog()
win.show()
app.exec()

print(win.selectedFiles())
print(win.result())
    

['C:/Users/Max/Desktop/MultiCarb3D/CAD2RAPID/500mm.gcode']
1


In [9]:
hi = {}
bye = {}
hi["test"] = 'lol'
hi["lol"] = 'test'
bye["test"] = 'lol'

print(hi != bye)

True


In [19]:
desk = os.environ['USERPROFILE']
logpath = desk / Path("Desktop/PRINT_log")
print(logpath)

C:\Users\Max\Desktop\PRINT_log


In [11]:
entry = QEntry(ID=0, PT='Q')
txt = "MoveL [[1.2,3.45,6.7],[0.0,0.0,0.0,0.0],[80,90,80,90]],z100,tool0"

try:
    entry.MT = re.findall('^Move[J,L,C]', txt, 0)[0][4]
    
    res_coor = re.findall('\d+\.\d+',txt)
    print(res_coor)

    entry.COOR_1.X = float(res_coor[0])
    entry.COOR_1.Y = float(res_coor[1])
    entry.COOR_1.Z = float(res_coor[2])
    entry.COOR_1.X_ori = float(res_coor[3])
    entry.COOR_1.Y_ori = float(res_coor[4])
    entry.COOR_1.Z_ori = float(res_coor[5])
    entry.COOR_1.Q = float(res_coor[6])

    print(re.findall('\d+,\d+,\d+,\d+(?=\]\],z)',txt))
    res_speed = re.findall('\d+,\d+,\d+,\d+(?=\],z)',txt)[0]
    res_speed = re.findall('\d+',res_speed)
    print(res_speed)

    entry.SV.TS = res_speed[0]
    entry.SV.OS = res_speed[1]
    entry.SV.ACR = res_speed[2]
    entry.SV.DCR = res_speed[3]

    zone,res = reShort('z\d+',txt,10)
    entry.Z = float(zone)

    # for later, if tool is implemented
    tool = reShort(',[^,]*$',txt,'tool0')

except Exception as e:
    print(None,e)

print(entry,None)

['1.2', '3.45', '6.7', '0.0', '0.0', '0.0', '0.0']
['80,90,80,90']
None list index out of range
ID: 0   MT: L   PT: Q   || COOR_1:   X: 1.2   Y: 3.45   Z: 6.7   Rx: 0.0   Ry: 0.0   Rz: 0.0   Q: 0.0   EXT: 0.0   || COOR_2:   X: 0.0   Y: 0.0   Z: 0.0   Rx: 0.0   Ry: 0.0   Rz: 0.0   Q: 0.0   EXT: 0.0   || SV:   TS: 200   OS: 50   ACR: 50   DCR: 50   || SBT: 0   SC: V   Z: 10   || TOOL:   M1: 0, 0   M2: 0, 0   M3: 0, 0   PNMTC_CLAMP: 0, 0   KNIFE: 0, 0   M4: 0, 0   PNMTC_FIBER: 0, 0   TIME: 0, 0 None


In [2]:
print(Coor(1,1,1,1,1,1,1,1) + Coor(1,2,3,4,5,6,7,8))

X: 2   Y: 3   Z: 4   Rx: 5   Ry: 6   Rz: 7   Q: 8   EXT: 9


In [52]:
import math as m

txt = 'G1 X0 Y0\nG28 X0 Y0.2\nG1 X0 Y0\n'
rows = txt.split('\n')

X = 0.0
Y = 0.0
num = 0
length = 0.0

for row in rows:
    if ( len( re.findall('^G\d+', row) ) > 0 ):
        num += 1

    Y_new = float( reShort('Y\d+[,.]\d+', row, Y, 'Y\d+')[0] )
    X_new = float( reShort('X\d+[,.]\d+', row, X, 'X\d+')[0] )

    length += m.sqrt( m.pow( (X_new - X), 2 ) + m.pow( (Y_new - Y), 2 ) )

    X = X_new
    Y = Y_new

0.0 0.0
0.0 0.2
0.0 -0.2
0.0 0.0
3 0.4
